In [4]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [5]:
#create path
path = r'C:\Users\TypicalPancake\6.10 Gun Violence 12-2024'

In [6]:
#import dataframe
df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'gv_cleaned.csv'))

In [15]:
# Define the function to extract participant data
def extract_participant_data(df, incident_column, participant_columns):
    participant_data = []
    
    for _, row in df.iterrows():
        incident_id = row[incident_column]
        
        # Parse participant data
        participants = {
            col: row[col].split("||") if pd.notna(row[col]) else []
            for col in participant_columns
        }
        
        # Iterate through participants
        max_participants = max(len(participants[col]) for col in participant_columns if participants[col])
        for i in range(max_participants):
            participant_entry = {"Incident ID": incident_id}
            for col in participant_columns:
                try:
                    key, value = participants[col][i].split("::", 1)
                    participant_entry[col] = value
                except (IndexError, ValueError):
                    participant_entry[col] = None  # Handle missing values
            participant_data.append(participant_entry)
    
    return pd.DataFrame(participant_data)

# Define the columns containing participant data
participant_columns = [
    "participant_age",
    "participant_age_group",
    "participant_gender",
    "participant_name",
    "participant_relationship",
    "participant_status",
    "participant_type",
    "gun_type",
    "gun_stolen"
]

# Generate the participant output for the entire dataset
participant_output = extract_participant_data(df, "incident_id", participant_columns)

# Verify and export the result
if isinstance(participant_output, pd.DataFrame):
    print("Columns in the extracted DataFrame:")
    print(participant_output.columns.tolist())
    
    # Export to CSV
    participant_output.to_csv(os.path.join(path, '02 Data','Prepared Data',"participant_gun_table_full.csv"))
    print("Participant data exported to participant_table_full.csv")
else:
    print("Error: `participant_output` is not a DataFrame.")


Columns in the extracted DataFrame:
['Incident ID', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type', 'gun_type', 'gun_stolen']
Participant data exported to participant_table_full.csv


In [8]:
# Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(participant_output.head())


First few rows of the DataFrame:
   Incident ID participant_age participant_age_group participant_gender  \
0       461105              20             Adult 18+               Male   
1       461105            None             Adult 18+               Male   
2       461105            None             Adult 18+               Male   
3       461105            None             Adult 18+             Female   
4       461105            None             Adult 18+               None   

  participant_name participant_relationship participant_status  \
0      Julian Sims                     None           Arrested   
1             None                     None            Injured   
2             None                     None            Injured   
3             None                     None            Injured   
4             None                     None            Injured   

  participant_type gun_type gun_stolen  
0           Victim     None       None  
1           Victim     None       No

# Merging to gv_cleaned.csv

In [20]:
# Load your datasets
participant_gun_table = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'participant_gun_table_full.csv'))
gv_cleaned = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'gv_cleaned.csv'))
# Ensure column naming consistency
participant_gun_table.rename(columns={'Incident ID': 'incident_id'}, inplace=True)

# Identify the columns to replace in gv_cleaned
columns_to_replace = [
    'participant_age',
    'participant_age_group',
    'participant_gender',
    'participant_name',
    'participant_relationship',
    'participant_status',
    'participant_type',
    'gun_type',
    'gun_stolen'
]

# Drop duplicates in `participant_gun_table` based on the key
participant_gun_table_deduped = participant_gun_table.drop_duplicates(subset=['incident_id'])

# Replace overlapping columns in `gv_cleaned` using `incident_id` as the key
for column in columns_to_replace:
    if column in participant_gun_table_deduped.columns and column in gv_cleaned.columns:
        gv_cleaned.set_index('incident_id', inplace=True)
        participant_gun_table_deduped.set_index('incident_id', inplace=True)
        gv_cleaned.update(participant_gun_table_deduped[column])
        gv_cleaned.reset_index(inplace=True)
        participant_gun_table_deduped.reset_index(inplace=True)

# Save the updated dataset
gv_cleaned.to_csv('merged_gv_cleaned.csv', index=False)
